In [1]:
import pandas as pd

In [2]:
#importing asset report that you dl from your client's cms
asset = pd.read_csv('asset_report.csv')

In [3]:
asset_1 = asset.copy()

In [4]:
asset_1.columns

Index(['approx_daily_views', 'asset_id', 'asset_type', 'status',
       'metadata_origination', 'custom_id', 'season', 'episode_title', 'tms',
       'active_claims', 'constituent_asset_id', 'active_reference_id',
       'inactive_reference_id', 'match_policy', 'is_merged',
       'conflicting_owner', 'ownership', 'asset_label',
       'conflicting_country_code'],
      dtype='object')

In [5]:
asset_1 = asset_1.fillna(0)

In [6]:
#asset_2 is for later use during the assets mapping process.
asset_2 = asset_1[['asset_id','episode_title','asset_label']]

In [7]:
#extracting asset label column from the asset report,
#this is the column that contains relevant show/series names that you need for asset mapping.

asset_1_assetlabels = asset_1[['asset_label']]
asset_1_assetlabels['asset_label'] = asset_1_assetlabels['asset_label'].str.lower()

/Users/azhoula/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [8]:
asset_1_assetlabels.head()

,asset_label
0,reference|the paynes
1,where are they now
2,where are they now
3,raising whitley
4,where are they now


In [9]:
#removing any depulicated values
asset_1_assetlabels = asset_1_assetlabels.drop_duplicates()

In [10]:
#spliting any value that has a '|'
new = asset_1_assetlabels['asset_label'].str.split("|", expand = True)
new.head()

,0,1,2
0,reference,the paynes,None
1,where are they now,None,None
3,raising whitley,None,None
5,NaN,NaN,NaN
6,reference,the oprah winfrey show,None


In [11]:
asset_1_assetlabels['asset_labels1'] = new[0]
asset_1_assetlabels['asset_labels2'] = new[1]

In [12]:
#In my case, column 1 and 2 are the columns that need to be filter again (remove duplicated values)
new_df1 = asset_1_assetlabels[['asset_labels1']].drop_duplicates()
new_df1.count()

asset_labels1    150
dtype: int64

In [13]:
new_df2 = asset_1_assetlabels[['asset_labels2']].drop_duplicates()
new_df2.head()

,asset_labels2
0,the paynes
1,None
6,the oprah winfrey show
7,greenleaf
147,the haves and have nots


In [14]:
#combining the filtered/cleaned columns (df1 & df2)
frame = new_df1.append(new_df2, ignore_index=True)
frame = frame.fillna('none')
frame = frame['asset_labels1'].append(frame['asset_labels2'])
frame = pd.DataFrame(frame)
frame.columns = ['show_names']
frame.count()

/Users/azhoula/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


show_names    336
dtype: int64

In [15]:
#Now, you have a list of show/series names that you will be needed for next step, which is mapping.
frame = frame.drop_duplicates()
frame.head()

,show_names
0,reference
1,where are they now
2,raising whitley
3,none
4,e10


In [ ]:
#print(frame)

In [ ]:
#frames.to_csv('frame.csv')

import the claim report and merge it with asset_2 df that was created eariler, based on asset id.

In [16]:
#import claim report
claim = pd.read_csv('claim_report.csv')

/Users/azhoula/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
claim_1 = claim.copy()

In [18]:
#there are so many columns in the claim report, but we only need 2 columns - asset id and video title
claim_1 = claim_1[['asset_id','video_title']]

In [19]:
#merge asset_2 onto claim report
merger = claim_1.merge(asset_2, how = 'left', left_on = 'asset_id', right_on = 'asset_id' )

In [20]:
#you have created a new df that contains a complete list of asset ids that have active claim(s)
#these are the assets that you want to prioritize for your mapping project
merger = merger.fillna(0)
merger.head()

,asset_id,video_title,episode_title,asset_label
0,A233951392426168,Dr. Phil to Betrayed Wife: Don't Blame Yoursel...,Dr. Phil to Betrayed Wife: Don't Blame Yoursel...,The Oprah Winfrey Show
1,A233951392426168,Dr. Phil to Betrayed Wife: Dont Blame Yourself...,Dr. Phil to Betrayed Wife: Don't Blame Yoursel...,The Oprah Winfrey Show
2,A233951392426168,Dr. Phil to Betrayed Wife: Don't Blame Yoursel...,Dr. Phil to Betrayed Wife: Don't Blame Yoursel...,The Oprah Winfrey Show
3,A233951392426168,Dr. Phil to Betrayed Wife: Don't Blame Yoursel...,Dr. Phil to Betrayed Wife: Don't Blame Yoursel...,The Oprah Winfrey Show
4,A259145521198487,Deleted Scenes: Jarring Up Jelly with Betty | ...,20130223-tnd-105-web-1-making-jelly.mp4,Trouble Next Door


In [21]:
merger.count()

asset_id         54205
video_title      54205
episode_title    54205
asset_label      54205
dtype: int64

In [25]:
#scanning through all 3 columns (video_title, episode_title, and asset_label) against the extracted show names

df = pd.DataFrame()

for show in frame.show_names:
    
    master_df = pd.DataFrame()
    
    mask = merger['episode_title'].str.contains(show,case = False,na=False)
    master_df = master_df.append(merger[mask])
    master_df["show_names"] = show
    mask2 = merger['video_title'].str.contains(show,case = False,na=False)
    master_df = master_df.append(merger[mask2])
    master_df["show_names"] = show
    mask3 = merger['asset_label'].str.contains(show,case = False,na=False)
    master_df = master_df.append(merger[mask3])
    master_df["show_names"] = show
    
    df = df.append(master_df, ignore_index=True)

In [24]:
df.columns

Index(['asset_id', 'asset_label', 'episode_title', 'show_names',
       'video_title'],
      dtype='object')

In [26]:
df = df[['asset_id','show_names','asset_label','episode_title','video_title']]

In [27]:
df_1 = df.groupby(['asset_id','show_names'])

In [28]:
#df_1.head()
df_1.count()

asset_label  episode_title  \
asset_id     show_names                                               
 --x7RzGYx40 in the bedroom                        1              1   
             oprah winfrey network                 1              1   
 -H51xs7VEcE big bowl of love                      1              1   
             oprah winfrey network                 1              1   
 -KnPHWckOnc mystery diagnosis                     1              1   
             oprah winfrey network                 1              1   
 -QH13xxNQeQ big bowl of love                      1              1   
             oprah winfrey network                 1              1   
 -ajFelpKD7g big bowl of love                      1              1   
             oprah winfrey network                 1              1   
 -jK0zOdyz-E oprah winfrey network                 1              1   
             searching for                         1              1   
 -wTulV8z8Ls oprah winfrey network                 1              1   
             our america                           2              2   
             our america with lisa ling            2              2   
0E5ckTFo8Uw  oprah winfrey network                 1              1   
             searching for                         2              2   
0Ns9n46F93M  in the bedroom                        1              1   
             oprah winfrey network                 2              2   
0cfgCM5O8Og  in the bedroom                        2              2   
             oprah winfrey network                 3              3   
1JG4UdHbFlA  enough already peter walsh            1              1   
             oprah winfrey network                 1              1   
1Ztn1DrnMkk  in the bedroom                        4              4   
             oprah winfrey network                 2              2   
1x5YctQj2i4  miracle                               3              3   
             miracle detectives                    3              3   
             oprah winfrey network                 1              1   
1zhWlFW5jU4  in the bedroom                        2              2   
             oprah winfrey network                 1              1   
...                                              ...            ...   
wSDoaTG0VV4  mystery diagnosis                     2              2   
             oprah winfrey network                 1              1   
wWbCvk3asOU  oprah behind the scenes               1              1   
woqu3b08ufU  big bowl of love                      2              2   
xMj2GSgAQzc  addicted to food                      2              2   
             oprah winfrey network                 1              1   
xz6cZpUEDm0  oprah winfrey network                 1              1   
             why not                               1              1   
yFJzcMa3J0M  addicted to food                      3              3   
             oprah winfrey network                 1              1   
yJFg41zL8uI  oprah winfrey network                 2              2   
ySAXmBHJHJc  mystery diagnosis                     2              2   
             oprah winfrey network                 1              1   
z-JogmX1UCI  breaking down the bars                2              2   
             oprah winfrey network                 1              1   
zJ_HLSubEq4  in the bedroom                        2              2   
             oprah winfrey network                 1              1   
zNNd3EEaCHs  miracle                               2              2   
             miracle detectives                    2              2   
             oprah winfrey network                 1              1   
zQbFHRWcmPI  miracle                               2              2   
             miracle detectives                    2              2   
             oprah winfrey network                 1              1   
zT-Cn_ul564  oprah winfrey network                 3             

In [ ]:
ugc_df.to_excel('new.xlsx')